In [ ]:
import pandas as pd
import numpy as np

### Import and Process Data

In [ ]:
df0 = pd.read_csv("data generation/train csv/saved_train_dataframe0.csv")
df1 = pd.read_csv("data generation/train csv/saved_train_dataframe1.csv")
df2 = pd.read_csv("data generation/train csv/saved_train_dataframe2.csv")
df3 = pd.read_csv("data generation/train csv/saved_train_dataframe3.csv")
df4 = pd.read_csv("data generation/train csv/saved_train_dataframe4.csv")
df5 = pd.read_csv("data generation/train csv/saved_train_dataframe5.csv")


# Concatenate 
df_train = pd.concat([df0, df1, df2, df3, df4, df5], ignore_index=True)

# Standarize to 0.05 forward
df_train["error"] = df_train["F0"] - 0.05
df_train["F0"] = 0.05
df_train["K"] = (0.05 * df_train["K"]) / (0.05 + df_train["error"])
df_train["alpha"] = df_train["alpha"] * ((0.05 / (0.05 + df_train["error"])) ** (1 - df_train["beta"]))

# Add relative strike column
df_train['K_relative'] = df_train['K'] / df_train['F0']

# Remove unnecessary columns
df_train.drop(columns=['F0', 'K', 'price', 'beta'], inplace=True)

### Remove Bad Rows

In [12]:
# Mark the initially bad rows
bad_rows = df_train[
    (df_train['IV'] == 0) |
    (np.isinf(df_train['IV'])) |
    (df_train['IV'].isna())
].copy()

# Create a list of indexes to delete
indexes_to_delete = set(bad_rows.index)

# Expand the deletion around bad rows (this removes further unstable rows where too few paths reached the strike )
for idx in bad_rows.index:
    deleted_row = df_train.loc[idx]
    deleted_K = deleted_row['K_relative']
    alpha_rho_nu_T = (deleted_row['alpha'], deleted_row['rho'], deleted_row['nu'], deleted_row['T'])

    # Check rows before and after
    for offset in [-3, -2, -1, 1, 2, 3]:
        neighbor_idx = idx + offset

        # Make sure neighbor index is inside the dataframe
        if 0 <= neighbor_idx < len(df_train):
            neighbor_row = df_train.loc[neighbor_idx]

            # Check if alpha, rho, nu, T match
            if (neighbor_row['alpha'], neighbor_row['rho'], neighbor_row['nu'], neighbor_row['T']) == alpha_rho_nu_T:

                # Only delete rows with lower or higher moneyness, depending on if in or out the money
                if deleted_K < 1 and neighbor_row['K_relative'] < deleted_K:
                    indexes_to_delete.add(neighbor_idx)
                elif deleted_K > 1 and neighbor_row['K_relative'] > deleted_K:
                    indexes_to_delete.add(neighbor_idx)

# Drop all marked rows
df_train_cleaned = df_train.drop(index=indexes_to_delete).reset_index(drop=True)

print(f"Deleted {len(indexes_to_delete)} rows total.")

Deleted 79694 rows total.


In [13]:
# Save the cleaned dataframe
df_train_cleaned.to_csv('train_dataset.csv', index=False)